# STAT 542 / CS 598: Homework 2
Fall 2019, by Edgar Pino

Due: Monday, Sep 23 by 11:59 PM Pacific Time

# Question 1 [20 Points] Linear Model Selection

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import itertools
import time
import numpy as np
from sklearn.preprocessing import scale

In [2]:
np.random.seed(1)

In [ ]:
boston_df = pd.read_csv('./data/boston.csv')

In [ ]:
boston_df = boston_df.drop([boston_df.columns[0],'medv', 'town', 'tract'],axis=1)

In [668]:
y = boston_df.cmedv

In [669]:
x = boston_df.drop(['cmedv'],axis=1)

In [670]:
X2 = sm.add_constant(x)

/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [671]:
est = sm.OLS(y, X2)

In [672]:
est2 = est.fit()

In [673]:
boston_df.drop(['cmedv'],axis=1).columns

Index(['lon', 'lat', 'crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis',
       'rad', 'tax', 'ptratio', 'b', 'lstat'],
      dtype='object')

In [301]:
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:                  cmedv   R-squared:                       0.746
Model:                            OLS   Adj. R-squared:                  0.738
Method:                 Least Squares   F-statistic:                     95.82
Date:                Sun, 22 Sep 2019   Prob (F-statistic):          5.77e-135
Time:                        22:14:18   Log-Likelihood:                -1492.9
No. Observations:                 506   AIC:                             3018.
Df Residuals:                     490   BIC:                             3086.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -434.9958    303.210     -1.435      0.1

## 5A - [5 Points] Report the most significant variable from this full model with all features.

Answer: 

One of the most significant variables is `nox`.

### 5B - [5 Points] Starting from this full model, use stepwise regression with both forward and backward and BIC criterion to select the best model. Which variables are removed from the full model?

In [302]:
def diff(first, second):
        second = set(second)
        return [item for item in first if item not in second]

In [303]:
def forward_regression(x, y,p_threshold):
    initial_list = []
    included = list(initial_list)
    while True:
        changed=False
        excluded = list(set(x.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(x[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < p_threshold:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True

        if not changed:
            break

    return diff(list(x.columns), included)

In [304]:
def backward_regression(x, y, p_threshold):
    included=list(x.columns)
    while True:
        changed=False
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(x[included]))).fit()
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() 
        if worst_pval > p_threshold:
            changed=True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
        if not changed:
            break
    return diff(list(x.columns), included)

In [238]:
forward_regression(x, y, 0.05)

['lon', 'lat', 'indus', 'age']

In [239]:
backward_regression(x,y,0.05)

['lon', 'lat', 'indus', 'age']

Answer:

The features removed are: `lon, lat, indus, and age.`

## 5C - [5 Points] Starting from this full model, use the best subset selection and list the best model of each model size.

In [240]:
def processSubset(feature_set):
    model = sm.OLS(y,x[list(feature_set)])
    regr = model.fit()
    RSS = ((regr.predict(x[list(feature_set)]) - y) ** 2).sum()
    return {"model":regr, "RSS":RSS}

In [241]:
def getBestModel(k):
    tic = time.time()
    
    results = []
    
    for combo in itertools.combinations(x.columns, k):
        results.append(processSubset(combo))
    
    models = pd.DataFrame(results)
    
    best_model = models.loc[models['RSS'].values.argmin()]
    
    toc = time.time()
    print("Processed", models.shape[0], "models on", k, "predictors in", (toc-tic), "seconds.")
    
    return best_model

In [242]:
models_best = pd.DataFrame(columns=["RSS", "model"])

In [243]:
tic = time.time()
for i in range(1,15):
    models_best.loc[i] = getBestModel(i)

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

Processed 15 models on 1 predictors in 0.053468942642211914 seconds.
Processed 105 models on 2 predictors in 0.247955322265625 seconds.
Processed 455 models on 3 predictors in 1.0631449222564697 seconds.
Processed 1365 models on 4 predictors in 3.4864137172698975 seconds.
Processed 3003 models on 5 predictors in 7.438055753707886 seconds.
Processed 5005 models on 6 predictors in 12.801699876785278 seconds.
Processed 6435 models on 7 predictors in 17.11007785797119 seconds.
Processed 6435 models on 8 predictors in 18.334751844406128 seconds.
Processed 5005 models on 9 predictors in 16.593907356262207 seconds.
Processed 3003 models on 10 predictors in 8.68674111366272 seconds.
Processed 1365 models on 11 predictors in 4.113813161849976 seconds.
Processed 455 models on 12 predictors in 1.4867126941680908 seconds.
Processed 105 models on 13 predictors in 0.3564932346343994 seconds.
Processed 15 models on 14 predictors in 0.04661107063293457 seconds.
Total elapsed time: 92.51844930648804 se

In [244]:
models_best

,RSS,model
1,29423.376266,<statsmodels.regression.linear_model.Regressio...
2,15226.522528,<statsmodels.regression.linear_model.Regressio...
3,14181.164611,<statsmodels.regression.linear_model.Regressio...
4,13395.365780,<statsmodels.regression.linear_model.Regressio...
5,12980.289500,<statsmodels.regression.linear_model.Regressio...
6,12257.279438,<statsmodels.regression.linear_model.Regressio...
7,11931.141396,<statsmodels.regression.linear_model.Regressio...
8,11657.539412,<statsmodels.regression.linear_model.Regressio...
9,11464.331057,<statsmodels.regression.linear_model.Regressio...
10,11339.974458,<statsmodels.regression.linear_model.Regressio...


# Question 2 (50 Points) Code Your Own Lasso

In [746]:
n = 200
p = 200

In [747]:
V = np.full((p,p), 0.2)

In [748]:
np.fill_diagonal(V, 1)

In [749]:
mu = np.full(p, 0)

In [750]:
X = np.random.multivariate_normal(mu, V, p)

In [751]:
y = X[:,1] + 0.5*X[:,2] + 0.25*X[:,3] + np.random.normal(n)

In [752]:
X = scale(X)

In [753]:
y = scale(y)

## A [10 Points] Hence, we need first to write a function that updates just one parameter, which is also known as the soft-thresholding function. Construct the function in the form of soft_th <- function(b, lambda), where b is a number that represents the one-dimensional linear regression solution, and lambda is the penalty level. The function should output a scaler, which is the minimizer of

In [695]:
def soft_th(b, lambda_val):
    if b > 0.0 and lambda_val < abs(b):
        return b - lambda_val
    elif b < 0.0 and lambda_val < abs(b):
        return b + lambda_val
    else:
        return 0.0

## B [10 Points] Now lets pretend that at an iteration, the current parameter β value is given below (as beta_old, i.e., βold). Apply the above soft-thresholding function to update all p parameters sequencially one by one to complete one “loop” of the updating scheme. Please note that we use the Gauss-Seidel style coordinate descent, in which the update of the next parameter is based on the new values of previous entries. Hence, each time a parameter is updated, you should re-calculate the residual

In [684]:
b_beta_old = np.zeros(X.shape[1])
b_lambda_val = 0.7

In [700]:
for iteration in range(1):
    for j in range(0, len(b_beta_old)):
        tmp_beta = b_beta_old.copy()
        tmp_beta[j] = 0.0
        r_j = y - np.dot(X, tmp_beta)
        beta = np.dot(X[:, j], r_j)/200 
        
        b_beta_old[j] = soft_th(beta, b_lambda_val)

## C [25 Points] Now, let us finish the entire Lasso algorithm. We will write a function myLasso(X, y, lambda, tol, maxitr). Set the tolerance level tol = 1e-5, and maxitr = 100 as the default value. Use the “one loop” code that you just wrote in the previous question, and integrate that into a grand for-loop that will continue updating the parameters up to maxitr runs. Check your parameter updates once in this grand loop and stop the algorithm once the ℓ1 distance between βnew and βold is smaller than tol. Use beta_old = rep(0, p) as the initial value, and lambda = 0.3. After the algorithm converges, report the following: i) the number of iterations took; ii) the nonzero entries in the final beta parameter estimate, and iii) the first three observations of the residual. Please write your algorithm as efficient as possible.

In [754]:
def myLasso(X, y, lambda_, tol=1e-5, maxitr=100):
    beta_old = np.zeros(X.shape[1])
    X.shape[1]
    for iteration in range(maxitr):
        beta_new = beta_old.copy()
        for j in range(0, len(beta_old)):
            tmp_beta = beta_old.copy()
            tmp_beta[j] = 0.0
            r_j = y - np.dot(X, tmp_beta)
            beta = np.dot(X[:, j], r_j)/X.shape[0] 
            beta_old[j] = soft_th(beta, lambda_)
        
        l1_distance = np.sum(np.abs(beta_new - beta_old))
        
        if l1_distance < tol:
            break
    print(f'Iterations took: {iteration + 1}')
    print(f'Non-zero entries: {beta_old[np.where(beta_old != 0)]}')
    print(f'First three observations of the residual: {r_j[:3]}')
    return beta_old

In [755]:
my_betas = myLasso(X, y, 0.3)

Iterations took: 6
Non-zero entries: [0.57212653 0.14388902 0.00998072]
First three observations of the residual: [ 1.0960527  -0.19095239  0.5545304 ]


## D [5 Points] Now we have our own Lasso function, let’s check the result and compare it with the glmnet package. Note that for the glmnet package, their lambda should be set as half of ours. Comment on the accuracy of the algorithm that we wrote. Please note that the distance of the two solutions should not be larger than 0.005.

In [757]:
from sklearn import linear_model

In [758]:
clf = linear_model.Lasso(alpha=0.3, fit_intercept=False, max_iter=100, tol=1e-5)

In [759]:
clf.fit(X, y)

Lasso(alpha=0.3, copy_X=True, fit_intercept=False, max_iter=100,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=1e-05, warm_start=False)

In [760]:
print(f"Distance between my algorithm and sklearn: {np.sum(np.abs(my_betas - clf.coef_))}")
print(f"Non-zero entries from sklearn: {clf.coef_[np.where(clf.coef_ != 0)]}")
print(f'Non-zero entries from my algorithm: {my_betas[np.where(my_betas != 0)]}')

Distance between my algorithm and sklearn: 3.2612801348363973e-16
Non-zero entries from sklearn: [0.57212653 0.14388902 0.00998072]
Non-zero entries from my algorithm: [0.57212653 0.14388902 0.00998072]


Overall, the algorithm we implemented gives me the same results as the one in sklearn. 

# Question 3 (30 Points) Cross-Validation for Model Selection

## A [10 Points] Do the following to process the data:
- Read data into R
- Convert character variables into factors
- Remove Item_Identifier
- Further convert all factors into dummy variables

### Answer

In [6]:
walmart_pd = pd.read_csv('./data/Train.csv').fillna(0)

In [7]:
walmart_pd.head(5)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,0,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [8]:
category_types = ["Item_Fat_Content", "Item_Type", "Outlet_Identifier", "Outlet_Size", "Outlet_Location_Type", "Outlet_Type"]

In [9]:
walmart_pd[category_types] = walmart_pd[category_types].astype('category')

In [10]:
walmart_pd = walmart_pd.drop(columns=['Item_Identifier'])

In [11]:
# pd.get_dummies(walmart_pd.Item_Fat_Content, prefix='Item_Fat_Content')
for category in category_types:
    dummies = pd.get_dummies(walmart_pd[category], prefix=category)
    walmart_pd = walmart_pd.drop(columns=[category])
    walmart_pd = pd.concat([walmart_pd, dummies], axis=1)

In [12]:
walmart_pd.head(5)

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales,Item_Fat_Content_LF,Item_Fat_Content_Low Fat,Item_Fat_Content_Regular,Item_Fat_Content_low fat,Item_Fat_Content_reg,...,Outlet_Size_High,Outlet_Size_Medium,Outlet_Size_Small,Outlet_Location_Type_Tier 1,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
0,9.30,0.016047,249.8092,1999,3735.1380,0,1,0,0,0,...,0,1,0,1,0,0,0,1,0,0
1,5.92,0.019278,48.2692,2009,443.4228,0,0,1,0,0,...,0,1,0,0,0,1,0,0,1,0
2,17.50,0.016760,141.6180,1999,2097.2700,0,1,0,0,0,...,0,1,0,1,0,0,0,1,0,0
3,19.20,0.000000,182.0950,1998,732.3800,0,0,1,0,0,...,0,0,0,0,0,1,1,0,0,0
4,8.93,0.000000,53.8614,1987,994.7052,0,1,0,0,0,...,1,0,0,0,0,1,0,1,0,0


## B [20 Points] Use all variables to model the outcome Item_Outlet_Sales in its log scale. First, we randomly split the data into two parts with equal size. Make sure that you set a random seed so that the result can be replicated. Treat one as the training data, and the other one as the testing data. For the training data, perform the following:

### Anwser

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV

In [28]:
def mean_square_error(y_true, y_pred):
    return np.square(np.subtract(y_true,y_pred)).mean()

#### Lasso CV

In [14]:
X = walmart_pd.drop(columns=['Item_Outlet_Sales'])

In [15]:
y = np.log(walmart_pd.Item_Outlet_Sales)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42, shuffle=False)

In [17]:
lasso_reg = LassoCV(cv=5, random_state=42).fit(X_train, y_train)

In [18]:
print(f"Number of iterations {lasso_reg.n_iter_}")

Number of iterations 16


In [19]:
print(f"Chosen alpha {lasso_reg.alpha_}")

Chosen alpha 0.032691486313765394


In [20]:
print(f"Coefficients {lasso_reg.coef_}")

Coefficients [-0.00963834 -0.          0.00826499 -0.00357406  0.         -0.
  0.          0.         -0.          0.          0.         -0.
  0.         -0.         -0.          0.         -0.          0.
 -0.          0.         -0.         -0.          0.         -0.
  0.         -0.         -0.          0.         -0.         -0.
  0.0885484   0.         -0.          0.          0.         -0.
 -0.          0.         -0.         -0.          0.         -0.
 -1.72635422  0.         -0.          0.        ]


In [21]:
print(f"Intercept {lasso_reg.intercept_}")

Intercept 13.575482597672433


#### Ridge CV

In [22]:
ridge_reg = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1]).fit(X_train, y_train)

In [23]:
print(f"Chosen alpha {ridge_reg.alpha_}")

Chosen alpha 1.0


In [24]:
print(f"Coefficients {ridge_reg.coef_}")

Coefficients [-0.00106192 -0.18374247  0.00826125  0.0051879   0.03742948 -0.02086543
 -0.00480627  0.04023754 -0.05199532  0.03516361  0.07383991 -0.08062557
  0.03418649 -0.05387222 -0.0353696   0.02145001 -0.07421994  0.03339989
  0.00525117  0.10502117 -0.02779904 -0.0259748   0.00454258 -0.03826962
  0.02327594 -0.42288516  0.02343379  0.12101093 -0.01926802 -0.36507693
  0.39462831 -0.02037229  0.06478643  0.29533885 -0.0715959  -0.2370878
  0.02343379  0.30376439 -0.09011038 -0.14133399  0.16542507 -0.02409109
 -0.7879621   0.41260181 -0.01926802  0.39462831]


In [25]:
print(f"Intercept {ridge_reg.intercept_}")

Intercept -4.474174809403746


#### Test these four models on the testing data and report and compare the prediction accuracy

In [26]:
y_lasso_test_pred = lasso_reg.predict(X_test)

In [29]:
print(f"MSE of Lasso: {mean_square_error(y_test, y_lasso_test_pred)}")

MSE of Lasso: 0.3077212821466025


In [32]:
y_ridget_test_pred = ridge_reg.predict(X_test)

In [33]:
print(f"MSE of Lasso: {mean_square_error(y_test, y_ridget_test_pred)}")

MSE of Lasso: 0.28401661733213585
